In [ ]:
# change trained information here
experiment = 'patent_claim_iter26272_as128_scalar1.0_cycle-auto_prenc-start_wsTrue_lg-latent_attn_add_attn_beta1.0_reg-kld_attn_mode-none_ffn_option-parallel_ffn_enc_layer-8_dec_layer-12_zdim-512_optFalse_ftFalse_zrate-0.5_fb-1sd-42_5.24'
latent_size = 512
max_length = 400
batch_size = 40
top_k = 100
top_p = 0.95

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/innovae-revision/innovae-adavae/adavae/src')

!pip install transformers
!pip install tensorboardX ipdb
!nvidia-smi
!pip install sentence-transformers

In [ ]:
import pandas as pd
df1 = pd.read_parquet('/content/drive/MyDrive/innovae-revision/exports/latent/processed_512_2epoch.parquet')
df1.loc[:525456,'set'] = 'train'
df1.loc[525456:554648,'set'] = 'val'
df1.loc[554648:,'set'] = 'test'

df2 = pd.read_csv('/content/drive/MyDrive/innovae-revision/exports/sample/selected_patents.csv')
df3 = pd.merge(df1, df2, how='inner', left_on='patent_id', right_on='patent_num')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from collections import defaultdict
from scipy.stats.stats import pearsonr
from matplotlib import pyplot as plt

import torch, math, argparse, copy, re
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from argparse import ArgumentParser
from transformers import GPT2Tokenizer, GPT2LMHeadModel

from adapters.configuration_gpt2 import GPT2Config
from adapters.vae import GPT2Adapter, AdaVAEModel
from adapters.common import AdapterConfig
from adaVAE import compute_loss
from data import GenerationDataset, DataFrameDataset
from utils import init_para_frompretrained, tokenize, sample_sequence

<ipython-input-4-9e85c087a47d>:6: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [ ]:
parser = ArgumentParser()

# Default parameters are set based on single GPU training
parser.add_argument("--seed", type=int, default=42)

## mode options
parser.add_argument('--adapter_size', type=int, default=128,
                    help="Hidden size of GPT2 encoder/decoder adapter")
parser.add_argument('--latent_size', type=int, default=32,
                    help="Hidden size of latent code")
parser.add_argument('--encoder_n_layer', type=int, default=8,
                    help="attention layer number of GPT-2 encoder")
parser.add_argument('--decoder_n_layer', type=int, default=12,
                    help="attention layer number of GPT-2 decoder")
parser.add_argument('--class_num', type=int, default=2,
                    help="class number for controllable generation")
parser.add_argument('--adapter_scalar', type=str, default="1.0",
                    help="adapter scalar")
parser.add_argument('--ffn_option', type=str, default="parallel_ffn",
                    choices=['sequential', 'parallel_attn', 'parallel_ffn', 'pfeiffer'],
                    help="adapter type option")
parser.add_argument('--latent_gen', type=str, default="latent_attn",
                    help="method for encoder to latent space, averaged_attn for average attention from "
                         "TransformerCVAE, linear for taken the first encoder token to a linear like Optimus",
                    choices=['latent_attn', 'averaged_attn', 'linear', 'mean_max_linear'])
parser.add_argument('--attn_mode', type=str, default="none",
                    choices=['prefix', 'adapter', 'lora', 'none'],
                    help="attention transfer type")
parser.add_argument('--reg_loss', type=str, default="kld",
                    choices=['kld', 'adversarial', 'symlog'],
                    help="regularization loss for latent space")

## testing paramters
parser.add_argument('--batch_size', type=int, default=128,
                    help='batch size per GPU. Lists the schedule.')
parser.add_argument('--max_length', type=int, default=30,
                    help='max length of every input sentence')
parser.add_argument('--data-dir', type=str, default='data/optimus_dataset')
parser.add_argument('--out-dir', type=str, default='out')
parser.add_argument('--experiment', type=str, help="ckpt dirctory", default='out')
parser.add_argument('--adapter_init', type=str, default='bert', choices=['lora', 'bert', 'lisa', 'other'],
                    help="parameter initialization method for adapter layers.")
parser.add_argument('--workers', default=2, type=int, metavar='N',  help='number of data loading workers')
parser.add_argument("--total_sents", default=10, type=int, help="Total sentences to test recontruction/generation.")
parser.add_argument("--max_test_batch", default=10, type=int, help="Total sentence pairs to test interpolation/analogy.")
parser.add_argument("--num_interpolation_step", default=10, type=int)
parser.add_argument("--degree_to_target", type=float, default=1.0)
parser.add_argument("--max_val_batches", type=int, help="Max batch size number to test recontruction.", default=30)
parser.add_argument("--latest_date", type=str, help="Latest date for model testing.", default="")

## metrics
parser.add_argument('--au_delta', type=float, default=0.01,
                    help="threshold for activated unit calculation.")

# use GPU
parser.add_argument('--gpu', default=0, type=int)
parser.add_argument('--no_gpu', action="store_true")


# KL cost annealing, increase beta from beta_0 to 1 in beta_warmup steps
parser.add_argument('--beta_0', default=1.00, type=float)
parser.add_argument('--beta_warmup', type=int, default=1000)
parser.add_argument('--kl_rate', type=float, default=0.0)

# cyc_vae parameters
parser.add_argument('--cycle', type=int, default=2000)

## trigger
parser.add_argument('--load', action="store_true")
parser.add_argument('--save_all', action="store_true", help="save full parameters of the model")
parser.add_argument('--add_input', action="store_true")
parser.add_argument('--add_attn', action="store_true")
parser.add_argument('--add_softmax', action="store_true")
parser.add_argument('--add_mem', action="store_true")
parser.add_argument('--attn_proj_vary', action="store_true")
parser.add_argument('--finetune_enc', action="store_true")
parser.add_argument('--finetune_dec', action="store_true")
parser.add_argument('--weighted_sample', action="store_true")
parser.add_argument('--add_z2adapters', action="store_true")
parser.add_argument('--learn_prior', action="store_true")
parser.add_argument('--test_model', action="store_true")
parser.add_argument('--do_sample', action="store_true", help="sample for reconstruction")

args = parser.parse_args(f'--add_attn --weighted_sample --latent_size {latent_size} --max_length {max_length} --batch_size {batch_size} --experiment {experiment}'.split())

In [ ]:
# Set random seed
gpu = torch.cuda.is_available()
np.random.seed(args.seed)
prng = np.random.RandomState()
torch.random.manual_seed(args.seed)

if gpu:
    print("There are ", torch.cuda.device_count(), " available GPUs!")
    torch.cuda.set_device(args.gpu)
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    print('Current single GPU: {}'.format(torch.cuda.current_device()))
device = torch.device(args.gpu if torch.cuda.is_available() else "cpu")

# Load model and trained weights
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

config = GPT2Config()
ada_config = AdapterConfig(hidden_size=768,
                            adapter_size=args.adapter_size,
                            adapter_act='relu',
                            adapter_initializer_range=1e-2,
                            latent_size=args.latent_size,
                            class_num=args.class_num,
                            encoder_n_layer=args.encoder_n_layer,
                            decoder_n_layer=args.decoder_n_layer,
                            dis_emb=128,
                            init='other',
                            adapter_scalar=args.adapter_scalar,
                            ffn_option=args.ffn_option,
                            attn_mode=args.attn_mode,
                            latent_gen=args.latent_gen,
                            attn_option='none',
                            mid_dim=30,
                            attn_bn=25,
                            prefix_dropout=0.1,
                            tune_enc=False,
                            tune_dec=False,
                            add_z2adapters=args.add_z2adapters)

AdaVAE = AdaVAEModel(config, ada_config, add_input=args.add_input, add_attn=args.add_attn, add_softmax=args.add_softmax, add_mem=args.add_mem,
                attn_proj_vary=args.attn_proj_vary, learn_prior=args.learn_prior, reg_loss=args.reg_loss)

## load pre-trained weights
init_para_frompretrained(AdaVAE.transformer, gpt2_model.transformer, share_para=False)
init_para_frompretrained(AdaVAE.encoder, gpt2_model.transformer, share_para=False)
AdaVAE.lm_head.weight = gpt2_model.lm_head.weight
AdaVAE.eval()

## load trained parameters
print('Loading model weights...')
state = torch.load(os.path.join("./out", args.experiment, 'model_best_val.pt'))  # , map_location='cpu' model_latest.pt
if 'module' in list(state.keys())[0]:  # model_path is data parallel model with attr 'module'
    keys = copy.copy(state).keys()
    for k in keys:
        state[k.replace('module.', '')] = state.pop(k)

if not args.save_all:
    model_dict = AdaVAE.state_dict()
    additional_dict = {k: v for k, v in state.items() if k in model_dict}
    model_dict.update(additional_dict)
    AdaVAE.load_state_dict(model_dict)
else:
    AdaVAE.load_state_dict(state)
AdaVAE = AdaVAE.to(device)

There are  1  available GPUs!
Current single GPU: 0


Loading model weights...


In [ ]:
endoftext = tokenizer.convert_tokens_to_ids("<|endoftext|>")

def decode_sent(sent):
    sent = sent[sent.index(endoftext) + 1:]
    if endoftext in sent:
        sent = sent[:sent.index(endoftext)]
    return tokenizer.decode(sent).strip()

In [ ]:
AAPL = df3[df3['ticker'].isin(['AMZN'])]
mean_aapl = AAPL[[i for i in AAPL.columns if 'z' in i or 'var' in i]].mean()[:512].tolist()
var_aapl = AAPL[[i for i in AAPL.columns if 'z' in i or 'var' in i]].mean()[512:].tolist()
vector_aapl = (np.array(mean_aapl) + (0*np.exp(np.array(var_aapl))**(1/2))).tolist()
vector_aapl = torch.tensor(vector_aapl,device = 'cuda').reshape(1,512)

In [ ]:
with torch.no_grad():
  sents, _ = sample_sequence(AdaVAE,
                             args.max_length,
                             z = vector_aapl,
                             batch_size = 1,
                             top_k=200,
                             top_p=0.95,
                             device=device,
                             sample=False,
                             temperature=0.2,
                             eos_token=endoftext)

  #Sample sentences
  sents = sents.tolist()
  decoded = [decode_sent(sent) for sent in sents]
  print(decoded[0])

Year: 2012

Title: Automated configuration of a computing device

Claim: A method comprising: receiving, at a computing device, a request to perform a first task on a computing device; determining, by the computing device, a plurality of attributes of the computing device; determining, by the computing device, a second task on the computing device based on the plurality of attributes of the computing device; and providing, by the computing device, a user interface for the first task to a user of the computing device, wherein the user interface includes a plurality of attributes of the computing device, wherein the user interface includes a plurality of attributes of the computing device, and wherein the user interface includes a plurality of attributes of the computing device.                                                                                                                              �


In [ ]:
 if True:
    all = []
    for i in tqdm(range(3584),'Reconstructing:'):
      start = i*40
      end = (i+1)*40
      latent_means = df3.iloc[start:end,6:518].values.tolist()
      latent_means = torch.tensor(latent_means,device = 'cuda')
      with torch.no_grad():

            sents, _ = sample_sequence(AdaVAE,
                                       args.max_length,
                                       z=latent_means,
                                       batch_size=args.batch_size,
                                       top_k=top_k,
                                       top_p=top_p,
                                       temperature = 0.2,
                                       device=device,
                                       sample=True,
                                       eos_token=endoftext)

            # Sample sentences
            sents = sents.tolist()
            decoded = [decode_sent(sent) for sent in sents]
            df3.loc[start:(end-1),'reconstructed'] = decoded
            all += decoded
df3.to_csv('/content/drive/MyDrive/innovae-revision/reconstruction_selected_comp.csv',index = False)